In [1]:
import pandas as pd
import numpy as np

# <b>Importing Data <b>

In [2]:
df_branch_service = pd.read_json("branch_service_transaction_info.json")

## DATA PROFILING

## INITIAL SHAPE AND UNIQUE ROWS

In [3]:
print("Total rows and columns: ",df_branch_service.shape)
print("Total Number of unique rows: ", df_branch_service['txn_id'].nunique())

Total rows and columns:  (130653, 4)
Total Number of unique rows:  62354


## Getting the key columns

In [4]:
print(df_branch_service.columns)

Index(['txn_id', 'branch_name', 'service', 'price'], dtype='object')


## Total number of rows with missing/null values

In [5]:
df_totalnulls = df_branch_service.isnull().any(axis=1).sum()
empty_string_count = (df_branch_service == '').any(axis=1).sum()
empty_string_count += (df_branch_service == 'N/A').any(axis=1).sum()
print("Total number of rows with a null or missing value: ", df_totalnulls + empty_string_count)

Total number of rows with a null or missing value:  33007


## Total Number of Exact Duplicates (Ground for Duplication)

In [6]:
print(df_branch_service.duplicated().sum())

12377


## Total Number of Duplicated Transaction IDs  (One-Many Relationship)

In [7]:
print(df_branch_service.duplicated(subset=['txn_id']).sum())

68299


## List of All Values per Column

In [8]:
print(df_branch_service['branch_name'].unique())
print(df_branch_service['service'].unique()) 
print(df_branch_service['price'].unique()) 

['MallOfAsia' 'Starmall' 'SmallMall' 'MayMall' 'FrankMall' 'Megamall'
 'RobinsonsMall' '' None 'N/A']
['Manicure' 'HairColor' 'FootSpa' 'Rebond' 'Haircut' 'NailColor'
 'Pedicure']
[         nan   0.          30.1237897   66.12345678  77.987989
 100.12123    400.23123     55.2324      88.09393   ]


## Frequency Distributions per Column

### Transaction ID

In [9]:
df_branch_service['txn_id'].describe().to_frame()

,txn_id
count,130653
unique,62354
top,TXN-06185
freq,18


### Service

In [10]:
df_branch_service['service'].describe().to_frame()

,service
count,130653
unique,7
top,NailColor
freq,19034


In [11]:
df_branch_service['service'].value_counts().to_frame()

,count
service,
NailColor,19034
Rebond,18776
Pedicure,18688
FootSpa,18676
Manicure,18648
HairColor,18537
Haircut,18294


### Branch Names

In [12]:
df_branch_service['branch_name'].describe().to_frame()

,branch_name
count,120345
unique,9
top,SmallMall
freq,16830


In [13]:
df_branch_service['branch_name'].value_counts().to_frame()

,count
branch_name,
SmallMall,16830
Megamall,16651
MallOfAsia,16641
MayMall,16572
FrankMall,16532
RobinsonsMall,16520
Starmall,16308
N/A,2159
,2132


### Price

In [14]:
df_branch_service['price'].describe()

count    110653.000000
mean         95.966969
std         115.874117
min           0.000000
25%          30.123790
50%          66.123457
75%          88.093930
max         400.231230
Name: price, dtype: float64

## List of all Prices per Service

In [15]:
print(df_branch_service.groupby('service')['price'].unique())

service
FootSpa        [nan, 0.0, 100.12123]
HairColor       [nan, 0.0, 88.09393]
Haircut      [nan, 0.0, 66.12345678]
Manicure         [nan, 0.0, 55.2324]
NailColor     [nan, 0.0, 30.1237897]
Pedicure       [nan, 0.0, 77.987989]
Rebond         [nan, 0.0, 400.23123]
Name: price, dtype: object


## Count of Services Provided per Branch

In [16]:
service_list = df_branch_service.groupby('branch_name')['service'].nunique()
print(service_list)

branch_name
                 1
FrankMall        7
MallOfAsia       7
MayMall          7
Megamall         7
N/A              1
RobinsonsMall    7
SmallMall        7
Starmall         7
Name: service, dtype: int64


## Null Checking of Branch Name (for removal)

In [17]:
filtered_branch = df_branch_service[(df_branch_service['branch_name'] == 'N/A') | (df_branch_service['branch_name'] == '') 
| (df_branch_service['branch_name'].isna())]
print(filtered_branch)

           txn_id branch_name    service       price
21      TXN-43150                Haircut         NaN
24      TXN-59988        None     Rebond         NaN
50      TXN-25855         N/A  NailColor         NaN
66      TXN-08569        None     Rebond         NaN
75      TXN-03974        None     Rebond         NaN
...           ...         ...        ...         ...
130602  TXN-39344        None   Manicure   55.232400
130607  TXN-60320        None     Rebond  400.231230
130618  TXN-43035        None  HairColor   88.093930
130625  TXN-37684                Haircut   66.123457
130646  TXN-43876                Haircut   66.123457

[14599 rows x 4 columns]


## Null Checking for Price (for fixing. One-One for service and price)

In [18]:
filtered_price = df_branch_service[(df_branch_service['price'] == 0)] 
print(filtered_price)

          txn_id    branch_name    service  price
20000  TXN-05451        MayMall  HairColor    0.0
20001  TXN-05451       Starmall  HairColor    0.0
20002  TXN-02255      FrankMall     Rebond    0.0
20003  TXN-02255       Megamall     Rebond    0.0
20004  TXN-02255        MayMall     Rebond    0.0
...          ...            ...        ...    ...
39995  TXN-45930       Starmall    Haircut    0.0
39996  TXN-03162  RobinsonsMall   Pedicure    0.0
39997  TXN-03162        MayMall   Pedicure    0.0
39998  TXN-16572       Megamall   Pedicure    0.0
39999  TXN-16572      SmallMall   Pedicure    0.0

[20000 rows x 4 columns]


In [20]:
filtered_price = (df_branch_service[df_branch_service['price'].isna()])
print(filtered_price)

          txn_id branch_name    service  price
0      TXN-24546  MallOfAsia   Manicure    NaN
1      TXN-14642    Starmall  HairColor    NaN
2      TXN-60295   SmallMall    FootSpa    NaN
3      TXN-60295    Starmall    FootSpa    NaN
4      TXN-60295     MayMall    FootSpa    NaN
...          ...         ...        ...    ...
19995  TXN-11721        None  HairColor    NaN
19996  TXN-64348  MallOfAsia    Haircut    NaN
19997  TXN-64348  MallOfAsia    Haircut    NaN
19998  TXN-64348  MallOfAsia    Haircut    NaN
19999  TXN-05451    Megamall  HairColor    NaN

[20000 rows x 4 columns]


## Null Checking of Service (for removal)

In [21]:
filtered_branch = df_branch_service[(df_branch_service['service'] == 'N/A') | (df_branch_service['service'] == '') 
| (df_branch_service['service'].isna())]
print(filtered_branch)

Empty DataFrame
Columns: [txn_id, branch_name, service, price]
Index: []


# END OF PROFILING

### REMOVE DUPLICATES


In [ ]:
print(df_branch_service.shape)
df_branch_service = df_branch_service.drop_duplicates()
print(df_branch_service.shape)



### Remove Duplicate Transaction IDs

In [ ]:
df_branch_service.drop_duplicates(subset=['txn_id'], inplace=True)

print(df_branch_service.shape)
print(df_branch_service.nunique())

In [ ]:
df_branch_service.to_parquet('branch_service_duplicates_removed.parquet')

### REMOVE NULL VALUES

In [ ]:
df_branch_service = df_branch_service.dropna(subset=['branch_name'])
df_branch_service = df_branch_service.dropna(subset=['service'])
print('Null Values Dropped Currently at:',df_branch_service.shape)


### REMOVE / REPLACE MISSING VALUES

In [ ]:
df_branch_service = df_branch_service.drop(df_branch_service[df_branch_service['branch_name'] == ''].index)
df_branch_service = df_branch_service.drop(df_branch_service[df_branch_service['branch_name'] == 'N/A'].index)

# conditions to fill in missing prices
serviceArray = ['Manicure', 'HairColor', 'FootSpa', 'Rebond', 'Haircut', 'NailColor', 'Pedicure']
priceArray = [55.23, 88.09, 100.12, 400.23, 66.12, 30.12, 77.99]

for i in range(len(serviceArray)):
    df_branch_service.loc[(df_branch_service['service'] == serviceArray[i]) & (df_branch_service['price'].isnull()), 'price'] = priceArray[i]


print(df_branch_service.shape)
df_branch_service.to_parquet('branch_service_nullbranch_removed.parquet')

### Formatting Values

### CHANGE VALUES TO CAMELCASE

In [ ]:
df_branch_service.loc[df_branch_service['branch_name'] == 'Starmall', 'branch_name'] = 'StarMall'
df_branch_service.loc[df_branch_service['branch_name'] == 'Megamall', 'branch_name'] = 'MegaMall'

### ROUND  PRICES TO 2 DECIMAL PLACES

In [ ]:
df_branch_service = df_branch_service.round({'price': 2})

In [ ]:
df_branch_service.to_parquet('branch_service_formatted_values.parquet')

In [ ]:
df_branch_service.sort_values(by=['txn_id'], inplace=True, ascending=True)     
df_branch_service.head(30)